# Define Functions and Run Initial Query

In [2]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import json

pd.options.display.float_format = '{:,.5f}'.format
pd.options.display.max_columns = 500
pd.options.display.max_rows = 20

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    TestGroup = np.array(('364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275')).astype(int)
    df['TestGroup'] = (df['Employee'].isin(TestGroup)).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    df['Proactive Launched']=(df['Date']>=datetime(2023,11,10)).astype(int)
    df['Constant'] =1
    return df

In [20]:
dfp = run_query(
"""
SELECT
    Metric
FROM
    hive.care.expert_performance_metrics
GROUP BY METRIC
""")
dfp

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_4916\494237659.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,expert_id,Metric,metric_level,numerator_name,denominator_name,numerator,denominator,Date,client,subclient,icp_Client,LOB,channel_name,operator,dm_source,year_month,Month,metrics_group,Block,emp_sk,OS,site,username,Expert,Coach,manager,tenure_group,metric_lob,MetricType,supervisor_id,manager_id
0,579450,checkout claims,true,checkout_claims,,3.00000,None,2023-10-29,verizon wireless,verizon wireless,MOB-Verizon,mobility,voice,add,all,2023-10,2023-10-01,,,,,bears,mariacamila.heredia,maría heredia,julieth andrea almeida,charlie escobar,121-180,,,568497,524531


In [23]:
vhe_dfp = run_query(
"""
select UPPER(metric), MIN(Date), MAX(DATE)
FROM 
     hive.care.expert_performance_metrics
GROUP BY 
UPPER(metric)
""")

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_4916\494237659.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


In [24]:
vhe_dfp
# vhe_dfp[].values_count()

,_col0,_col1
0,2022-09-01,2023-12-12


In [ ]:
df = vhe_dfp.copy()
json_df = pd.DataFrame.from_records(df['edp_raw_data_map'].apply(json.loads))
display(vhe_dfp.head(),json_df.head(),json_df.columns)
display(pd.DataFrame(json_df['ExtraData_inputMessage'].value_counts()).sort_values('ExtraData_inputMessage',ascending=False).head(20))
# json_df.to_excel('JsonData.xlsx')
del df, json_df


In [ ]:
home_df = run_query(
"""
SELECT
element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') AutoFeedback,
element_at(VHE.edp_raw_data_map, 'Name')
FROM 
    hive.care.l1_verizon_home_events VHE
WHERE 1=1
    AND element_at(VHE.edp_raw_data_map, '_header_eventContext_producer') = 'eip-ingestion-data-science'
    AND element_at(VHE.edp_raw_data_map, 'Identities_messageSid') IS NOT NULL
    --AND element_at(VHE.edp_raw_data_map, 'Name') = 'RequestSummaryVoice'
    AND element_at(VHE.edp_raw_data_map, 'Scope')='GenerativeAISearchBotVoice'
    AND (element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') in ('That worked!', 'Still need help', 'Wrong issue') or element_at(VHE.edp_raw_data_map, 'ExtraData_inputMessage') is null)
    AND (CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) in (573190, 573276, 572909, 573192, 573585, 573573, 572815, 552121, 576911, 572247)
    OR CAST(element_at(VHE.edp_raw_data_map, 'ExtraData_agentId') AS INT) is null)
    AND DATE(edp_updated_date) > DATE('2023-11-10')
    
""")
home_df

In [ ]:
df=run_query(file_to_string(r'G:\Decision_Support\Reporting\Business Insights\Power BI\Generative AI Dashboards\Expert Assist Pilot\AdHocs\8. Finding Proactive Messages\ExpertAssistResponsesByType.sql'))


In [ ]:
df[['Proactive Messages','Reactive Messages']].sum()